**Google Colab Notebook - Final Submission**

---

This notebook implements real-time dynamic parking pricing using the **Pathway** library with the `dataset.csv` file provided. It includes:
- Real-time data streaming with Pathway
- Feature engineering and demand calculation
- Three pricing models
- Bokeh visualization support

---

In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import datetime
import random
import pathway as pw
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import column

output_notebook()

## 📥 Step 1: Load & Preprocess Dataset

In [4]:
df = pd.read_csv("dataset.csv")
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S', errors='coerce')
df = df.sort_values('Timestamp').reset_index(drop=True)

# Save for Pathway stream simulation
df[['Timestamp', 'SystemCodeNumber', 'Occupancy', 'Capacity', 'QueueLength',
    'VehicleType', 'TrafficConditionNearby', 'IsSpecialDay']].to_csv("parking_stream.csv", index=False)

## 🧱 Step 2: Define Schema & Stream

In [6]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    VehicleType: str
    TrafficConditionNearby: str
    IsSpecialDay: int

stream = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=100)

## ⚙️ Step 3: Feature Engineering

In [8]:
fmt = "%Y-%m-%d %H:%M:%S"

enriched = stream.with_columns(
    ts = stream.Timestamp.dt.strptime(fmt),
    occupancy_ratio = stream.Occupancy / stream.Capacity,

    traffic_level = pw.apply(
        lambda s: {"low": 1, "medium": 2, "average": 2, "high": 3}.get(str(s).lower(), 2),
        stream.TrafficConditionNearby
    ),

    vehicle_weight = pw.apply(
        lambda s: {"bike": 0.5, "car": 1.0, "truck": 1.5}.get(str(s).lower(), 1.0),
        stream.VehicleType
    ),
)


## 💰 Step 4: Pricing Models & Demand

In [12]:
# Step 1: Compute demand using pw.apply
enriched = enriched.with_columns(
    demand = pw.apply(
        lambda o, q, t, s, v: (
            1.0 * o +
            0.5 * q -
            0.3 * t +
            1.5 * s +
            0.8 * v
        ),
        enriched.occupancy_ratio,
        enriched.QueueLength,
        enriched.traffic_level,
        enriched.IsSpecialDay,
        enriched.vehicle_weight
    )
)

# Step 2: Compute prices based on demand
enriched = enriched.with_columns(
    price_model1 = pw.apply(lambda x: round(10 + 2 * x, 2), enriched.occupancy_ratio),

    price_model2 = pw.apply(
        lambda d: round(min(20, max(5, 10 * (1 + d / 10))), 2),
        enriched.demand
    ),

    price_model3 = pw.apply(
        lambda d: round(min(25, max(5, 10 * (1 + d / 10) * random.choice([0.9, 1.0, 1.1]))), 2),
        enriched.demand
    ),

    reroute_flag = enriched.Occupancy >= enriched.Capacity
)


## 📤 Step 5: Select Output Columns

In [13]:
output = enriched.select(
    enriched.ts,
    enriched.SystemCodeNumber,
    enriched.Occupancy,
    enriched.Capacity,
    enriched.price_model1,
    enriched.price_model2,
    enriched.price_model3,
    enriched.reroute_flag,
)

## 🚀 Step 6: Run the Real-Time Pipeline


## 📑 Report Section

### ✅ Demand Function
The demand function is defined as:
```python
demand = 1.0 * occupancy_ratio +
         0.5 * QueueLength -
         0.3 * traffic_level +
         1.5 * IsSpecialDay +
         0.8 * vehicle_weight
```
### 🔧 Assumptions
- `traffic_level`: low=1, medium/average=2, high=3
- `vehicle_weight`: bike=0.5, car=1.0, truck=1.5
- Demand influences price in Model 2 & 3.

### 💵 Pricing Model Behavior
- **Model 1**: Linear w.r.t. occupancy.
- **Model 2**: Demand-based, bounded between 5–20.
- **Model 3**: Adds competitive randomness to Model 2.

### 📈 Visual Justification
Use Bokeh to compare models visually over time per parking space.


In [14]:
%%capture --no-display
pw.run()

Output()

In [15]:
# 📊 Bokeh Visualization: Static Preview from parking_stream.csv or any DataFrame

import pandas as pd
import random
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import column

output_notebook()

# Load dataset
df = pd.read_csv("parking_stream.csv")

# Preprocess timestamps
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Feature engineering (same as Pathway logic)
df['occupancy_ratio'] = df['Occupancy'] / df['Capacity']
df['traffic_level'] = df['TrafficConditionNearby'].str.lower().map({
    "low": 1, "medium": 2, "average": 2, "high": 3
})
df['vehicle_weight'] = df['VehicleType'].str.lower().map({
    "bike": 0.5, "car": 1.0, "truck": 1.5
})
df['demand'] = (
    1.0 * df['occupancy_ratio'] +
    0.5 * df['QueueLength'] -
    0.3 * df['traffic_level'] +
    1.5 * df['IsSpecialDay'] +
    0.8 * df['vehicle_weight']
)

# Pricing models
df['price_model1'] = round(10 + 2 * df['occupancy_ratio'], 2)
df['price_model2'] = (10 * (1 + df['demand'] / 10)).clip(5, 20).round(2)
df['price_model3'] = (df['price_model2'] * df['price_model2'].apply(lambda x: random.choice([0.9, 1.0, 1.1]))).clip(5, 25).round(2)

# Plot for a single parking lot
lot_id = df['SystemCodeNumber'].unique()[0]
lot_data = df[df['SystemCodeNumber'] == lot_id].sort_values("Timestamp")
source = ColumnDataSource(lot_data)

# Bokeh plot
p = figure(
    x_axis_type="datetime",
    title=f"Pricing Models Over Time for Lot: {lot_id}",
    height=350, width=800
)
p.line('Timestamp', 'price_model1', source=source, color='blue', legend_label='Model 1 (Linear)', line_width=2)
p.line('Timestamp', 'price_model2', source=source, color='green', legend_label='Model 2 (Demand-based)', line_width=2)
p.line('Timestamp', 'price_model3', source=source, color='red', legend_label='Model 3 (Competitive)', line_width=2)

# Add hover tooltip
p.add_tools(HoverTool(
    tooltips=[("Time", "@Timestamp{%F %T}"), ("Price", "$y")],
    formatters={"@Timestamp": "datetime"}
))
p.legend.location = "top_left"

# Show plot
show(p)


In [16]:
# 📊 Bokeh Multi-Lot Pricing Dashboard

import pandas as pd
import random
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import column

output_notebook()

# Load and preprocess the dataset
df = pd.read_csv("parking_stream.csv")
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Feature engineering
df['occupancy_ratio'] = df['Occupancy'] / df['Capacity']
df['traffic_level'] = df['TrafficConditionNearby'].str.lower().map({
    "low": 1, "medium": 2, "average": 2, "high": 3
})
df['vehicle_weight'] = df['VehicleType'].str.lower().map({
    "bike": 0.5, "car": 1.0, "truck": 1.5
})
df['demand'] = (
    1.0 * df['occupancy_ratio'] +
    0.5 * df['QueueLength'] -
    0.3 * df['traffic_level'] +
    1.5 * df['IsSpecialDay'] +
    0.8 * df['vehicle_weight']
)

# Pricing models
df['price_model1'] = round(10 + 2 * df['occupancy_ratio'], 2)
df['price_model2'] = (10 * (1 + df['demand'] / 10)).clip(5, 20).round(2)
df['price_model3'] = (df['price_model2'] * df['price_model2'].apply(lambda x: random.choice([0.9, 1.0, 1.1]))).clip(5, 25).round(2)

# Create a Bokeh plot for each unique parking lot
plots = []

for lot_id in df['SystemCodeNumber'].unique():
    lot_data = df[df['SystemCodeNumber'] == lot_id].sort_values("Timestamp")
    source = ColumnDataSource(lot_data)

    p = figure(
        x_axis_type="datetime",
        title=f"Pricing Models - Lot {lot_id}",
        height=300,
        width=800
    )

    p.line('Timestamp', 'price_model1', source=source, color='blue', legend_label='Model 1 (Linear)', line_width=2)
    p.line('Timestamp', 'price_model2', source=source, color='green', legend_label='Model 2 (Demand-based)', line_width=2)
    p.line('Timestamp', 'price_model3', source=source, color='red', legend_label='Model 3 (Competitive)', line_width=2)

    p.add_tools(HoverTool(
        tooltips=[("Time", "@Timestamp{%F %T}"), ("Price", "$y")],
        formatters={"@Timestamp": "datetime"}
    ))

    p.legend.location = "top_left"
    plots.append(p)

# Display all plots vertically
show(column(*plots))
